<h1>
🦜⛓️‍💥LangChain
</h1>


# **Brief Recap**

**LangChain** is an open source framework for building applications based on large language models (LLMs). LangChain provides tools and abstractions to improve the customization, accuracy, and relevancy of the information the models generate. For example, developers can use LangChain components to build new prompt chains or customize existing templates. LangChain also includes components that allow LLMs to access new data sets without retraining.

With LangChain, organizations can repurpose LLMs for domain-specific applications without retraining or fine-tuning. Development teams can build complex applications referencing proprietary information to augment model responses. For example, you can use LangChain to build applications that read data from stored internal documents and summarize them into conversational responses.

# **Architecture**

<img src='assets/arch.png' width=450>

Here's an architecture and workflow of a LangChain-powered document processing and question-answering system. Let me break down its key components:

* **Input Processing**
  * The system starts with PDF documents on the left side
  * These documents are split into multiple chunks of text/documents
  * Each chunk goes through an embedding process (represented by binary code icons)
* **Data Processing Flow**
  * The text chunks are converted into embeddings (vector representations)
  * These embeddings are stored in a Vector Store (shown as a database icon)
  * The system uses specific technologies like:
    * Amazon Aurora
    * PostgreSQL with pgvector
    * Knowledge Base functionality
* **Query Processing**
  * A user inputs a question ("What is a neural network?")
  * The question goes through Question Embedding
  * A Semantic Search is performed against the vector store
  * The search produces Ranked Results
* **Response Generation**
  * The ranked results are processed by an LLM (Language Learning Model)
  * The LLM generates the final Answer back to the user

LangChain combines document processing, vector embeddings, and language models to create a comprehensive question-answering system.




# **Use Cases**

* **Document Processing and Analysis**
  * Parsing complex documents to extract structured information into JSON or tables
  * Automated extraction of dates, quantities, and transaction details from financial documents
  * Document classification using few-shot prompting
  * Processing and analyzing large volumes of text documents
* **Question Answering Systems**
  * Building chatbots and virtual assistants for customer support
  * Creating retrieval-based QA systems with document context
  * Implementing conversational agents that can access and reference internal documents
  * Developing SQL-based QA systems that work with various database dialects
* **Content Generation and Summarization**
  * Generating executive summaries of documents and meeting notes
  * Creating summaries of large documents using map-reduce techniques
  * Producing concise summaries of financial reports and earnings documents
  * Translating content across multiple languages
* **Conversational AI**
  * Context-aware chatbots for customer support
  * Virtual agents that can access and reference company documentation
  * Automated appointment scheduling and customer service systems

* **Code-Related Tasks**
  * Code analysis for detecting bugs and security vulnerabilities
  * Development of coding assistants to improve programmer productivity
  * Custom development environments with integrated LLM capabilities



---



# **Implementation**


There are majorly 3 main components in LangChain:

1. **Components**
  * **LLM Wrappers**: There are LLM wrappers that allows us to connect to the LLMs like GPT4.
  * **Prompt Templates**: They allow us to avoid hard code text the input to the LLMs.
  * **Indexes**: Allow us to extract relevant info from the LLMs. For eg: PineCone VectorStore.

2. **Chains**
  * They allow us to combine multiple components together to solve a specific task building an entire LLM application

3. **Agents**
  * It allows the LLMs to interact with external APIs.

We will be unpacking these concepts discussing and implementing each of them.





### **Initial Setup**
Firstly we are going to pip install the following libraries:
```python
python-dotenv==1.0.0
langchain==0.0.137
pinecone-client==2.2.1
```
* **python-dotenv**: To manage env file with passwords
* **pinecone-client**: Vector store database

### **API Keys**

Sign up and gather your API keys from all the following websites:
* [Pinecone](https://www.pinecone.io/?utm_term=pinecone%20database&utm_campaign=brand-us-e&utm_source=adwords&utm_medium=ppc&hsa_acc=3111363649&hsa_cam=21023369441&hsa_grp=167470667468&hsa_ad=690982708943&hsa_src=g&hsa_tgt=kwd-1627713670725&hsa_kw=pinecone%20database&hsa_mt=e&hsa_net=adwords&hsa_ver=3&gad_source=1&gclid=CjwKCAiAxqC6BhBcEiwAlXp453Gr9f08CPnE8_0JZfjvb1y_D0fb7uDaO-3btwyVnvrvI_tkfawjVBoCMeUQAvD_BwE)

* [OpenAI](https://platform.openai.com/docs/overview)


In [ ]:
# Install Dependencies

!pip install python-dotenv==1.0.0
!pip install langchain==0.0.137
!pip install pinecone-client==2.2.1

In [ ]:
# Save your API keys in an env variable
%%writefile .env

OPENAI_API_KEY="YOUR API KEY"
PINECONE_API_KEY="YOUR API KEY"
PINECONE_ENV="YOUR API KEY"

In [ ]:
# Load environment variables

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

### **LLM Wrapper**

In this section, we're going to see how LangChain allows us to interact with LLMs.

In [ ]:
# Run basic query with OpenAI wrapper

from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")
llm("explain large language models in one sentence")

* It takes in a "text-davinci-003" LLM.
* The output is something similar to when you run it by the OpenAI API directly.


In [ ]:
# import schema for chat messages and ChatOpenAI in order to query chatmodels GPT-3.5-turbo or GPT-4

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [ ]:
# Initializing the LLM
chat = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.3)

# List of messages to send to the chat model
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data ")
]

# Response is stored here
response=chat(messages)

print(response.content,end='\n')

**Breakdown**

1. **Initializing the ChatOpenAI instance:**

  ```python
  chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

  ```

* This creates an instance of the `ChatOpenAI` class, which is a wrapper for interacting with OpenAI's chat models.
* `model_name="gpt-3.5-turbo"` specifies that we're using the "gpt-3.5-turbo" model.
* `temperature=0.3` sets the temperature parameter to 0.3, which controls the randomness of the model's output. A lower temperature like 0.3 makes the output more focused and deterministic, while a higher temperature makes it more creative and unpredictable.

2. **Defining the messages:**

```python
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data ")
]
```

* This creates a list of messages that will be sent to the chat model.
* **`SystemMessage`** provides overall instructions or context to the model. Here, it's telling the model to act as an "expert data scientist."
* **`HumanMessage`** represents the user's input or query. In this case, it's asking the model to "Write a Python script that trains a neural network on simulated data."

**In essence,**

This sets up a conversation with the "gpt-3.5-turbo" chat model, provides it with a system prompt and a user query, and then prints the model's response.



### **Prompt Templates**

Prompt templates are pre-defined recipes for generating prompts to feed to Language Models (LLMs). Instead of hardcoding the entire prompt string every time, you can use templates with placeholders (variables) that are filled in later. This makes your prompts more flexible and reusable.

LangChain provides the `PromptTemplate` class for creating and managing these templates.

In [ ]:
# Import prompt and define PromptTemplate

from langchain import PromptTemplate

template = """
You are an expert data scientist with an expertise in building deep learning models.
Explain the concept of {concept} in a couple of lines
"""

prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)

# Run LLM with PromptTemplate

llm(prompt.format(concept="autoencoder"))

**Breakdown**

1. **Prompt Creation**
* `input_variables=["concept"]`: This specifies the name of the variable(s) that will be used in the template.
* `template=template`: This assigns the template string you defined earlier.

2. **Using the Prompt**
* `prompt.format(concept="autoencoder")`: This fills in the {concept} placeholder with the value "autoencoder", generating the complete prompt string.
* `llm(...)`: This sends the formatted prompt to the LLM (which you initialized earlier) to get a response.

**In essence,**

* It creates a reusable template for asking the LLM to explain a data science concept.
* Provides a way to easily change the concept being asked about without rewriting the entire prompt.
* Makes the code more organized and easier to understand.


### **Chains**


In this section we're going to see how to link multiple LangChain components together to create more complex workflows. This is useful for building applications where you want to process information in stages or combine the outputs of different components.

LangChain provides the `LLMChain` class for creating chains.


**Initializing Chain**

* We will first define a chain using the language model (`llm`) and prompt (`prompt`) as arguments.
* This chain combines the OpenAI language model (`llm`) with the previously defined prompt (`prompt`) that asks for an explanation of a data science concept.

In [ ]:
# Import LLMChain and define chain with language model and prompt as arguments.

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("autoencoder"))

**Defining a Sequential Chain**

* Then we define a second prompt (`second_prompt`) that asks for a simplified explanation of the concept, as if explaining it to a five-year-old.

In [ ]:
# Define a second prompt

second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Turn the concept description of {ml_concept} and explain it to me like I'm five in 500 words",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

* Now we import `SimpleSequentialChain` and define a new chain (`overall_chain`) that combines the first two chains in sequence.

* This means that the output of the first chain (the initial explanation) will be used as input to the second chain (to generate the simplified explanation).

* Finally, we run the `overall_chain` by specifying only the input variable for the first chain (e.g., "autoencoder").

In [ ]:
# Define a sequential chain using the two chains above: the second chain takes the output of the first chain as input

from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
explanation = overall_chain.run("autoencoder")
print(explanation)



**In essence,**

* We saw how to create and run individual chains using `LLMChain`.
* It shows how to combine chains sequentially using `SimpleSequentialChain` to create more complex workflows.
* The sequential chain enables the output of one chain to be used as input to another, allowing for multi-step information processing.

### **Embeddings & VectorStores**

**Embeddings** are essentially numerical representations of text. They capture the semantic meaning of words and phrases by converting them into vectors (lists of numbers). Similar words and phrases have similar vector representations, allowing you to perform tasks like similarity search.

**Vectorstores** are databases specifically designed to store and search these vector representations (embeddings). They enable efficient retrieval of relevant information based on semantic similarity.
We will be using the **Pinecone** vector database.




**Text Splitting**

* We shall begin by importing `RecursiveCharacterTextSplitter` from LangChain for splitting text into smaller chunks.
* `text_splitter` is initialized with a chunk size of 100 characters and no overlap.
* `texts` is created by splitting the explanation variable into smaller documents using create_documents.

In [ ]:
# Import utility for splitting up texts and split up the explanation given above into document chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 0,
)

texts = text_splitter.create_documents([explanation])

In [ ]:
# Individual text chunks can be accessed with "page_content"

texts[0].page_content

**Embeddings**

* `OpenAIEmbeddings` is imported to create embeddings using OpenAI's models.
* `embeddings` is initialized using the "ada" model.

In [ ]:
# Import and instantiate OpenAI embeddings

from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model_name="ada")

**Query Embedding**

* `embed_query` is used to turn the first text chunk (`texts[0].page_content`) into a vector representation (embedding).
* The resulting embedding (`query_result`) is printed.

In [ ]:
# Turn the first text chunk into a vector with the embedding

query_result = embeddings.embed_query(texts[0].page_content)
print(query_result)

**Vectorstore (Pinecone)**

* We import `pinecone` and initialize the Pinecone client with API key and environment.
* `Pinecone.from_documents` is used to create a Pinecone index named `langchain-quickstart` and upload the text chunks (`texts`) with their embeddings.

In [ ]:

# Import and initialize Pinecone client

import os
import pinecone
from langchain.vectorstores import Pinecone


pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment=os.getenv('PINECONE_ENV')
)

In [ ]:
# Upload vectors to Pinecone

index_name = "langchain-quickstart"
search = Pinecone.from_documents(texts, embeddings, index_name=index_name)

**Similarity Search**

* A query is defined: `"What is magical about an autoencoder?"`
* `similarity_search` is used to search the Pinecone index for documents that are semantically similar to the query.
* The search results (`result`) are printed.

In [ ]:
# Do a simple vector similarity search

query = "What is magical about an autoencoder?"
result = search.similarity_search(query)

print(result)

**In essence,**

We learnt how to split text into chunks, create embeddings for these chunks using OpenAI's embedding models, store them in a Pinecone vectorstore, and perform a similarity search.

### **Agents**

We will learn how to use LangChain to create an agent that can execute Python code. This allows you to combine the power of LLMs with the ability to run code and interact with the outside world.

We will be importing the following classes to achieve this:


* `create_python_agent`: Used to create an agent specifically for executing Python code.
* `PythonREPLTool`: Provides the functionality to interact with a Python Read-Eval-Print Loop (REPL), allowing the agent to run Python code.
* `PythonREPL`: The actual Python REPL environment used by the tool.
* `OpenAI`: The language model used by the agent (OpenAI's models in this case).

In [ ]:
# Import Python REPL tool and instantiate Python agent

from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI


In [ ]:
agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True
)


**Breakdown**
* `create_python_agent` is called to create the agent.
* `llm=OpenAI(temperature=0, max_tokens=1000)`: Specifies the language model to use (OpenAI with temperature 0 and a maximum of 1000 tokens).
* `tool=PythonREPLTool()`: Provides the Python REPL tool to the agent.
* `verbose=True`: Enables verbose output, showing the agent's thought process and actions.

In [ ]:
# Execute the Python agent

agent_executor.run("Find the roots (zeros) if the quadratic function 3 * x**2 + 2*x -1")

**Breakdown**

* `agent_executor.run(...)` is used to execute the agent with a given task.
* The task in this case is to find the roots of the quadratic function.

**In essence,**

We created a Python agent that leverages an LLM (OpenAI) to understand instructions and a Python REPL tool to execute Python code. This enables the agent to solve tasks that require code execution, such as finding the roots of a quadratic equation. The verbose=True option provides insights into how the agent decides on actions and generates code to accomplish the given task.